In [2]:
import os
import pandas as pd

In [3]:
with open('dict/defdict.txt', 'r') as f:
    l = [line.strip('\n') for line in f.readlines()]

In [4]:
path = '/home/keae/Mould/data'
for file in os.listdir(path):
    df = pd.read_excel(os.path.join(path, file),
                       header=1, sheet_name='传送模设计科')
    df = df[['异常单编码', '品管异常描述']]
    df.insert(df.shape[1], '错误匹配', '')

    for index, row in df.iterrows():
        des = row['品管异常描述']
        for s in l:
            if des.find(s) != -1:
                df.loc[index, '错误匹配'] = df.loc[index, '错误匹配']+s+','
                # df.loc[index, '错误匹配'].strip(',')

    print(df)
    print(os.path.join(path, file))
    break


    异常单编码                                             品管异常描述  \
0   51389                 MGJ033-0106-0606-P1858压块设计未下图（未备料）   
1   51379  GL001-0202-P5821，P5822，P5823，刀口没有斜度.设计未标注，导致没加工斜度   
2   51329  MGJ033-0106-p1809A 平1812A 冲头与刀口对不上.3D的刀口处有个台阶。...   
3   51285                     GL001-0202-P7804镶块高度低于P7500镶块面   
4   51265                   标准件LY037-0205-S2018 无避位 （2件）设计错误   
5   51185             GL003-0202-P6008，因加工让位不够导致崩裂，烧焊返修后，新制。   
6   51146                       GL001-0202-P4502/P4503 设计漏下图   
7   51124  GL007-0202-P4077 P4078压板与GL007-0202-P4042上模导柱定...   
8   51122      CEN140-0707-P207A上模刀块。图档图层线是3D线，无法分解。此问题需要根治。   
9   51103                              GL001-0202-S4030 设计买错   
10  51085  GL001-0202-P7810 P7817 设计出错图，钳工要求设计出图软料新制不加工螺丝...   
11  51047                   MPC045-0102-P6809 设计出垫片不够导致加工量不足   
12  51031                        GL007-0102-P3308 有一螺丝孔设计漏标注   
13  51021                         MPC042-0101-P4807 与上模压料芯干涉   
14  51017              GL007-0102-P1509 